In [1]:
from types import SimpleNamespace
from pretrain.neuron import DMiner
from pretrain.dataset import get_dataloader

[WARNING]: failed to patch stdout/stderr for fork-safety: 'OutStream' object
has no attribute 'buffer'
[WARNING]: failed to reconfigure stdout/stderr with custom encoding error
handler: 'OutStream' object has no attribute 'reconfigure'
/Users/napoli/anaconda3/envs/311/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
a = SimpleNamespace()
a.config = DMiner.config()
dataloader = get_dataloader(a)

Found cached dataset wikitext (/Users/napoli/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126)
100%|██████████| 3/3 [00:00<00:00, 420.36it/s]


In [7]:
seen = set()
total = 0
for global_step, batch in enumerate( dataloader[0] ):
    batch = {k: v for k, v in batch.items()}
    seen.update( batch['id'].tolist() )
    if global_step == 10: break

In [9]:
len( seen )

176